In [ ]:
import time

from hypernodes import Pipeline, node
from hypernodes.engines import DaftEngine
from hypernodes.telemetry import ProgressCallback


@node(output_name="doubled")
def double(x: int) -> int:
    time.sleep(0.1)  # Small delay to see progress
    return x * 2


@node(output_name="result")
def add_one(doubled: int) -> int:
    time.sleep(0.1)
    return doubled + 1


# Create engine with progress callback
engine = DaftEngine(callbacks=[ProgressCallback(enable=True)])
pipeline = Pipeline(
    nodes=[double, add_one],
    name="double_and_add",
    engine=engine,
)

result = pipeline.run(inputs={"x": 5})
print(f"Result: {result}")

Result: {'doubled': 10, 'result': 11}


In [ ]:
"""Test ProgressCallback with .map() operation."""


@node(output_name="squared")
def square(x: int) -> int:
    # time.sleep(0.05)
    return x**2


# Create pipeline with progress for map
progress_map = ProgressCallback(enable=True)
pipeline_map = Pipeline(
    nodes=[square], engine=DaftEngine()
)  # , callbacks=[progress_map], name="map_example"
# ,


results = pipeline_map.map(inputs={"x": [1, 2, 3, 4, 5]}, map_over="x")
print(f"Results: {results}")


Results: [{'squared': 1}, {'squared': 4}, {'squared': 9}, {'squared': 16}, {'squared': 25}]


In [31]:
result = pipeline.run(inputs={"x": 5})
result

{'doubled': 10, 'result': 11}

In [38]:
import daft


@daft.func.batch(return_dtype=daft.DataType.int32())
def square(x: int) -> int:
    return x**2


In [39]:
%%time
df = daft.from_pydict({"x": list(range(20))})
df.with_column("squared", square(daft.col("x"))).collect()

Error when running pipeline node UDF square-4b8cc672-a89d-43ea-bd75-b799262b9a4e


CPU times: user 2.93 ms, sys: 3.88 ms, total: 6.81 ms
Wall time: 5.51 ms


TypeError: unsupported operand type(s) for ** or pow(): 'Series' and 'int'

In [ ]:
%%time
df = daft.from_pydict({"x": list(range(20))})
df.with_column("squared", square(daft.col("x"))).collect()

CPU times: user 2.68 ms, sys: 3.33 ms, total: 6.01 ms
Wall time: 4.1 ms


xInt64,squaredInt64
0,0
1,1
2,4
3,9
4,16
5,25
6,36
7,49


In [34]:
result = pipeline.map(inputs={"x": list(range(20))}, map_over="x")
result

[{'doubled': 0, 'result': 1},
 {'doubled': 2, 'result': 3},
 {'doubled': 4, 'result': 5},
 {'doubled': 6, 'result': 7},
 {'doubled': 8, 'result': 9},
 {'doubled': 10, 'result': 11},
 {'doubled': 12, 'result': 13},
 {'doubled': 14, 'result': 15},
 {'doubled': 16, 'result': 17},
 {'doubled': 18, 'result': 19},
 {'doubled': 20, 'result': 21},
 {'doubled': 22, 'result': 23},
 {'doubled': 24, 'result': 25},
 {'doubled': 26, 'result': 27},
 {'doubled': 28, 'result': 29},
 {'doubled': 30, 'result': 31},
 {'doubled': 32, 'result': 33},
 {'doubled': 34, 'result': 35},
 {'doubled': 36, 'result': 37},
 {'doubled': 38, 'result': 39}]

In [ ]:
from hypernodes import Pipeline, node
from hypernodes.telemetry import ProgressCallback


@node(output_name="doubled")
def double(x: int) -> int:
    time.sleep(0.1)  # Small delay to see progress
    return x * 2


@node(output_name="result")
def add_one(doubled: int) -> int:
    time.sleep(0.1)
    return doubled + 1


# Create pipeline with progress callback
engine = SeqEngine(callbacks=[ProgressCallback(enable=True)])
pipeline = Pipeline(
    nodes=[double, add_one],
    name="double_and_add",
    engine=engine,
)

result = pipeline.run(inputs={"x": 5})

In [ ]:
import logfire

from hypernodes import Pipeline, SeqEngine, node
from hypernodes.telemetry import TelemetryCallback

# Configure logfire (local only, no cloud export)
logfire.configure(send_to_logfire=False)

engine = SeqEngine(callbacks=[TelemetryCallback()])
pipeline = Pipeline(nodes=[double, add_one], name="double_and_add", engine=engine)

result = pipeline.run(inputs={"x": 5})

16:19:15.097 pipeline:pipeline_4541554880
16:19:15.098   node:double
16:19:15.205   node:add_one


In [7]:
chart = telemetry.get_waterfall_chart()
chart